In [14]:
import sqlite3
import pandas as pd
import os
os.getcwd()


'c:\\Users\\15025\\Documents\\CodeYou\\Capstone\\Notebooks'

In [10]:
# Create a connection to a new or existing database file
connection = sqlite3.connect("tornado_tracks.db")

print("Connected:", connection)

# Always close your connection when you're done
##connection.close()

Connected: <sqlite3.Connection object at 0x00000193DC7C4E50>


#### Load tables from CSV files

In [ ]:
weather_detail = pd.read_csv("../Datasets/weather_detail_cleaned.csv")
weather_detail.to_sql("weather_detail", connection, if_exists="replace", index=False)


In [ ]:
tornado_tracks = pd.read_csv("../Datasets/tornado_tracks_combined_cleaned.csv")
tornado_tracks.to_sql("tornado_tracks", connection, if_exists="replace", index=False)